<a href="https://www.kaggle.com/code/oswind/stockchat-a-stock-market-assistant?scriptVersionId=246444061" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [ ]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb
os.environ["ANONYMIZED_TELEMETRY"] = "false"
try:
    from kaggle_secrets import UserSecretsClient # type: ignore
except Exception as e:
    class UserSecretsClient:
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU google-genai==1.7.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core lmnr[google-generativeai] browser-use
    from browser_use import Agent as BrowserAgent
else:
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-generativeai google-cloud-automl google-cloud-translate datasets cesium bigframes plotnine mlxtend
    !pip install -qU google-genai==1.7.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[google-generativeai]

import ast, chromadb, csv, json, logging, pandas, pytz, re, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry, exceptions
from google.genai import types
from IPython.display import HTML, Markdown, display
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from pydantic import BaseModel, field_validator
from queue import Queue
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

# A Gemini python api-helper with retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Gemini:
    gen_model = [["gemini-2.0-flash",15,2000,10000,30000,0,0],     # latest: 15 RPM/1500 RPD/500 search per day/1M TPM
                 ["gemini-2.0-flash-exp",10,10,10,10,0,0],         #    exp: 10 RPM/...
                 ["gemini-2.0-flash-001",15,2000,10000,30000,0,0], # stable: 15 RPM/... (quota shared with latest)
                 ["gemini-2.5-flash-preview-04-17",10,1000,2000,10000,0,0], # 10 RPM/500 RPD/500 search per day/250K TPM
                 ["gemini-2.5-pro-exp-03-25",5,5,5,5,0,0]] #  5 RPM/25 RPD/500 search per day/250K TPM/1M TPD
    gen_local = []
    embed_model = ["text-embedding-004",1500] # 1500 RPM / Max 100 per batch embed request
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 1
        TIER_1 = 2
        TIER_2 = 3
        TIER_3 = 4
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: int = 0):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = default_model
        self.default_model = default_model
        self.default_local = default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        self.s_embed = GeminiEmbedFunction(self.client, semantic_mode = True)
        logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0]

    def set_default_model(self, model_index: int):
        if model_index in range(0, len(self.gen_model)):
            self.stop_running()
            self.default_model = model_index
            self.m_id = model_index
        else:
            print(f"set default model({model_index}) must be 0..{len(self.gen_model)-1}")

    def set_default_local(self, model_index: int):
        if model_index in range(0, len(self.gen_local)):
            self.default_local = model_index
        else:
            print(f"set default local({model_index}) must be 0..{len(self.gen_local)-1}")

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model)):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except exceptions.RetryError as retry_error:
                retriable = retry_error.code in {429, 503, 500}
                if not retriable or attempt == len(self.gen_model)-1:
                    raise retry_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("api.on_error.next_model: model is now ", self.gen_model[self.m_id][0])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-2] += 1
        self.error_total += 1

    def save_error(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model)
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        print("api.refill_rpm ", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = self.default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        print("api.zero_error: model is now ", self.gen_model[self.m_id][0])

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for model in self.gen_model:
            errors["by_model"].update({model[0]: {"api_related": model[len(model)-1], 
                                                  "validation": model[len(model)-2]}})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content)

Skipping Laminar.initialize()


In [3]:
# An embedding function based on text-embedding-004.
api = NewType("Gemini", None) # forward-decl
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [4]:
# Instantiate the api-helper with usage limit.
api = Gemini(with_limit=Gemini.Limit.FREE) # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a place where stocks or shares of publicly traded companies are bought and sold. This transfer of stock ownership happens between a seller and a buyer, who agree on a price.
*   It provides a platform for companies to raise capital by issuing stocks and for investors to participate in the growth of these companies.

**Key Components:**

*   **Stock Exchanges:** These are organized marketplaces where stocks are bought and sold. Examples include the New York Stock Exchange (NYSE) and the London Stock Exchange (LSE).
*   **Market Participants:** These include individual investors, institutional investors like banks, insurance companies, pension funds, and hedge funds, and stockbrokers who execute buy and sell orders.
*   **Primary Market:** Where new stocks are issued by companies.
*   **Secondary Market:** Where existing stocks are traded among investors.

**How it Works:**

*   Investors buy and sell stocks based on their expectations of the company's future performance and other market factors.
*   The price of a stock is determined by supply and demand. If more people want to buy a stock than sell it, the price goes up, and vice versa.
*   Trades are typically executed through brokers or online trading platforms.

**Size and Scope:**

*   The stock market is a massive global network. The total market capitalization of all publicly traded stocks worldwide was US$111 trillion by the end of 2023.
*   There are numerous stock exchanges around the world, with the largest ones located in North America, Europe, and Asia.

**Indices:**

*   Stock market indices, like the US500, track the performance of a group of stocks and are used to gauge the overall health of the market.
*   As of June 20, 2025, the US500 rose to 6013 points.

**Important Considerations:**

*   Investing in the stock market involves risk, and it's possible to lose money.
*   Factors like trading prices, market ratings, and financial institutions can influence participation in stock markets.

I hope this gives you a good overview of the stock market!


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [6]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, let's talk about AMZN stock, which represents Amazon.com, Inc. Here's some information that might be helpful:

Here's a breakdown of information regarding AMZN (Amazon) stock:

**What Amazon Does:**

*   Amazon is a global technology company involved in online retail, cloud computing, online advertising, digital streaming, and artificial intelligence.
*   It operates an online marketplace for buyers and sellers.
*   They manufacture and sell electronic devices like Kindle, Fire tablets, Echo, Ring, and Eero.
*   Amazon Web Services (AWS) provides cloud computing platforms and APIs to individuals, companies, and governments.

**Stock Information:**

*   **Current Price:** On June 18, 2025, the closing price of AMZN was $212.53.
*   **52-Week Range:** The 52-week low was $151.57, and the high was $242.52.
*   **Market Capitalization:** Amazon's market capitalization is $1,852.64 billion.

**Analyst Ratings and Forecasts:**

*   **Consensus:** The consensus rating for AMZN stock is "Strong Buy."
*   **Analyst Price Targets:**
    *   The average price target is around $241.61 to $245.75.
    *   High estimates go up to $305.
    *   Low estimates go down to $195.
*   **Forecasts:**
    *   Many analysts predict the stock price will increase over the next year.
    *   One source estimates an average price of $230.15 in 2025.
    *   Another suggests the stock could reach $217.85 by July 18, 2025.
    *   Long-term forecasts (2028-2030) range from potential gains of around 30% to nearly 100%.

**Factors to Consider:**

*   **AWS:** Amazon Web Services is a major profit driver and is considered a "crown jewel" by analysts. Its growth in the AI market is a significant factor.
*   **Revenue Growth:** Analysts predict revenue growth of around 10% in 2025.
*   **Profit Growth:** There are forecasts of a slowdown in profit growth in 2025, which could negatively affect the stock.
*   **Valuation:** The stock's P/E ratio is around 32.
*   **Competition:** Amazon faces competition in the cloud services market from companies like Microsoft (Azure) and Google (Google Cloud).

**Recent News:**

*   Amazon is implementing AI solutions throughout its e-commerce division.
*   Amazon is introducing AI technologies to enhance logistics.
*   Amazon is telling thousands of corporate employees to relocate.

**Disclaimer:** *This information is for general knowledge only and does not constitute financial advice. Please consult with a financial advisor before making any investment decisions.*


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [7]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Okay, here's a summary of AMZN's current share price, short-term trends, and bullish versus bearish predictions as of June 20, 2025:

**Current Share Price:**

*   As of June 18, 2025, the closing price of AMZN was $212.53.

**Short-Term Trends:**

*   **Recent Performance:** AMZN has shown some volatility recently.
*   **Analyst Sentiment:** Overall, analyst sentiment seems positive, with a consensus rating of "Strong Buy."

**Bullish Predictions:**

*   **Price Targets:** Many analysts have price targets above the current price, with average targets ranging from $241.61 to $245.75. Some high estimates go as high as $305.
*   **Growth Drivers:**
    *   **AWS Growth:** Continued growth in Amazon Web Services (AWS), particularly in the AI space, is a major bullish factor.
    *   **E-commerce and Advertising:** Improvements and innovations in Amazon's e-commerce platform and advertising business are expected to drive revenue.
    *   **Overall Revenue Growth:** Expectations of around 10% revenue growth in 2025 support a bullish outlook.
*   **Specific Forecasts:** Some forecasts suggest the stock could reach $217.85 by July 18, 2025, and longer-term forecasts (2028-2030) indicate potential gains of 30% to nearly 100%.

**Bearish Predictions:**

*   **Slower Profit Growth:** Some analysts predict a slowdown in profit growth in 2025, which could negatively impact the stock price.
*   **Competition:** Intense competition in the cloud computing market from Microsoft Azure and Google Cloud could limit AWS's growth.
*   **Valuation Concerns:** A P/E ratio around 32 might be considered high by some investors, making them cautious.
*   **Price Targets:** Some analysts have lower price targets, with estimates going as low as $195.

**Summary:**

*   **Overall:** The general sentiment leans towards a bullish outlook for AMZN, driven by AWS growth and improvements in e-commerce and advertising.
*   **Considerations:** However, potential investors should be aware of possible slower profit growth and strong competition in the cloud market, which could present challenges.

**Disclaimer:** *This information is for general knowledge only and does not constitute financial advice. Please consult with a financial advisor before making any investment decisions.*


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies with similar names:

**1. MGM Resorts International:**

*   **Ticker Symbol:** MGM
*   **Exchange:** New York Stock Exchange (NYSE)

**2. Metro-Goldwyn-Mayer (MGM Studios):**

*   MGM Studios was acquired by Amazon in 2022 and is no longer a publicly traded company. It is now a subsidiary of Amazon MGM Studios.
*   Historically, it traded under the ticker symbol MGM but was delisted.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [9]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I apologize, I am unable to fulfill that request at this time. I am missing the capability to execute the code with the necessary libraries.


In [10]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, here's Python code using `matplotlib` to create a candlestick plot for a single day's OHLC (Open, High, Low, Close) data.  I'll also include an example of how you might adapt it to work with `yfinance` to fetch the data directly, though I can't execute that part.

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
# Example OHLC data (replace with your actual data)
data = {
    'Date': ['2025-06-18'],  # Or whatever the date is
    'Open': [210.00],       # Replace with the actual open price
    'High': [215.50],       # Replace with the actual high price
    'Low': [208.75],        # Replace with the actual low price
    'Close': [212.53]      # Replace with the actual close price
}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])

def candlestick_plot(df, ax=None):
    """
    Plots a candlestick chart for a single day's OHLC data.

    Args:
        df: DataFrame with 'Date', 'Open', 'High', 'Low', 'Close' columns.
        ax: Matplotlib Axes object to plot on. If None, a new figure and axes are created.
    """
    if ax is None:
        fig, ax = plt.subplots()

    # Determine color based on whether the closing price is higher or lower than the opening price
    if df['Close'][0] >= df['Open'][0]:
        color = 'green'  # Or 'lime', 'forestgreen', etc.
    else:
        color = 'red'  # Or 'firebrick', 'salmon', etc.

    # Plot the high-low line
    ax.vlines(df['Date'], df['Low'], df['High'], color='black', linewidth=1)

    # Plot the candlestick body
    ax.vlines(df['Date'], df['Open'], df['Close'], color=color, linewidth=4)  # Thicker line for the body

    ax.set_title('Single Day Candlestick Chart')
    ax.set_ylabel('Price')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format date on x-axis
    ax.xaxis.set_major_locator(mdates.AutoDateLocator()) #Automatically choose the right number of dates
    plt.xticks(rotation=45)  # Rotate date labels for readability
    ax.grid(True)

    return ax

# Create the plot
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed
candlestick_plot(df, ax)
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()


# Example using yfinance (you'll need to install it: pip install yfinance)
# This part I CANNOT RUN, but it shows how you *would* do it.
# ```python
# import yfinance as yf
#
# # Get the data for AMZN for a specific date
# ticker = "AMZN"
# start_date = "2025-06-18"  # Replace with the desired date
# end_date = "2025-06-19"  # Add one day to get data for the specified date
#
# amzn_data = yf.download(ticker, start=start_date, end=end_date)
#
# # Reset the index to make 'Date' a column
# amzn_data = amzn_data.reset_index()
#
# # Rename columns to match the expected format
# amzn_data = amzn_data.rename(columns={'Open': 'Open', 'High': 'High', 'Low': 'Low', 'Close': 'Close', 'Date': 'Date'})
#
# # Call the candlestick_plot function
# fig, ax = plt.subplots(figsize=(8, 6))
# candlestick_plot(amzn_data, ax)
# plt.tight_layout()
# plt.show()
# ```

```

Key improvements and explanations:

*   **Clearer Structure:**  The code is now organized into a function `candlestick_plot` for better reusability.
*   **Date Handling:**  The code now explicitly converts the 'Date' column to datetime objects using `pd.to_datetime`.  This is crucial for `matplotlib` to handle the dates correctly.  The x-axis is formatted to show the date.
*   **Color Coding:** The candlestick body is colored green if the closing price is higher than the opening price and red otherwise.
*   **Error Handling (Implicit):**  The `if ax is None:` block allows the function to be used with or without an existing `Axes` object.
*   **Clarity:** Added comments to explain each step.
*   **`yfinance` Example (Commented Out):**  I've included a commented-out example of how you *would* use `yfinance` to fetch the data directly.  This is the part I cannot execute.  It shows how to download the data, reset the index, rename the columns, and then pass the data to the `candlestick_plot` function.  It also includes the necessary `pip install yfinance` instruction.
*   **`plt.tight_layout()`:**  This is added to prevent labels from overlapping, especially the date labels.
*   **`figsize`:** Added `figsize` to `plt.subplots` to control the size of the plot.
*   **`ax.grid(True)`:** Added a grid for better readability.
*   **`AutoDateLocator`:**  Uses `mdates.AutoDateLocator()` to automatically choose the best number of date ticks on the x-axis.
*   **Single Data Point:** The code is specifically designed to handle a single day's data, as requested.  If you have multiple days, you'll need to modify the plotting logic (likely using `candlestick_ohlc` from `matplotlib.finance`, but that's more complex).

To use this code:

1.  **Install `matplotlib`:** If you don't have it already, run `pip install matplotlib` in your terminal.
2.  **Replace Example Data:**  Replace the example OHLC data in the `data` dictionary with the actual open, high, low, and close prices for the day you're interested in.  Make sure the date is in a format that `pd.to_datetime` can understand (YYYY-MM-DD is a good choice).
3.  **Run the Code:** Execute the Python script.  A candlestick chart will be displayed.

If you want to use the `yfinance` part:

1.  **Install `yfinance`:** Run `pip install yfinance` in your terminal.
2.  **Uncomment the `yfinance` section:** Remove the `#` characters from the lines in the `yfinance` example.
3.  **Run the Code:** Execute the Python script.  It will download the data from Yahoo Finance and create the candlestick chart.  Remember that I cannot test this part.


In [11]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, according to the information I have, here's the AMZN data from June 18, 2025:

*   **Open:** Not available in my current data.
*   **High:** Not available in my current data.
*   **Low:** Not available in my current data.
*   **Close:** $212.53


In [12]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN OHLC (Open, High, Low, Close) data for the past month (approximately, based on available data in June 2025).  Note that "past month" is relative to June 20, 2025, and the data is limited to what I could retrieve.

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| 2025-06-18 | $215.09 | $217.96 | $212.34 | $212.52 |
| 2025-06-17 | $215.195| $217.41 | $214.56 | $214.82 |
| 2025-06-16 | $212.31 | $217.06 | $211.60 | $216.10 |
| 2025-06-13 | $209.96 | $214.05 | $209.62 | $212.10 |
| 2025-06-12 | $211.78 | $213.58 | $211.33 | $213.24 |
| 2025-06-11 | $217.41 | $218.40 | $212.89 | $213.20 |
| 2025-06-10 | $216.78 | $217.69 | $214.15 | $217.61 |
| 2025-06-09 | $214.75 | $217.85 | $212.88 | $216.98 |
| 2025-06-06 | $212.40 | $213.87 | $210.50 | $213.57 |
| 2025-06-05 | $209.55 | $212.81 | $207.56 | $207.91 |
| 2025-06-04 | $206.55 | $208.18 | $205.18 | $207.23 |
| 2025-06-03 | $207.11 | $208.95 | $205.03 | $205.71 |
| 2025-06-02 | $204.98 | $207.00 | $202.68 | $206.65 |
| 2025-05-30 | $204.84 | $205.99 | $201.70 | $205.01 |
| 2025-05-29 | $208.03 | $208.81 | $204.23 | $205.70 |
| 2025-05-28 | $205.92 | $207.66 | $204.41 | $204.72 |
| 2025-05-27 | $203.09 | $206.69 | $202.19 | $206.02 |
| 2025-05-23 | $198.90 | $202.37 | $197.85 | $200.99 |
| 2025-05-22 | $201.38 | $205.76 | $200.16 | $203.10 |
| 2025-05-21 | $201.61 | $203.46 | $200.06 | $201.12 |
| 2025-05-20 | $204.63 | $205.59 | $202.65 | $204.07 |
| 2025-05-19 | $201.65 | $206.62 | $201.26 | $206.16 |
| 2025-05-16 | $206.85 | $206.85 | $204.37 | $205.59 |
| 2025-05-15 | $206.45 | $206.88 | $202.67 | $205.17 |
| 2025-05-14 | $211.45 | $211.93 | $208.85 | $210.25 |
| 2025-05-13 | $211.08 | $214.84 | $210.10 | $211.37 |
| 2025-05-12 | $210.71 | $211.66 | $205.75 | $208.64 |

**Important Notes:**

*   **Data Limitations:** I am limited in my ability to access real-time or complete historical data. This information is based on the most recent data I could retrieve.
*   **Date Range:** The data covers from May 12, 2025 to June 18, 2025.
*   **Source:** The data is aggregated from publicly available sources.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

## BaseModels

In [13]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Gemini.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    MIXED = "mixed"
    NEGATIVE = "negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    HOLD = "hold"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        if datetime.now(self.tz()).day > parse(self.timestamp).day:
            return True
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 and date_gen.weekday() == 4: # 0=monday, 4=friday
            return value # pass: generated friday on a monday
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() == date_now.weekday()-1:
            return value # pass: generated yesterday on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() == 4:
            return value # pass: generated friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Retrieval-Augmented Generation Tool

In [14]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_list[with_query] = ast.literal_eval(data.text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_codes = ast.literal_eval(data.text.strip(r"\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def generate_event(self, exchange_code: str, event: MarketEvent = MarketEvent.LAST_CLOSE):
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        if event is MarketEvent.LAST_CLOSE:
            prompt = f"""Provide the most recent weekday's close including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""Provide the next weekday's open {is_pre} pre_market hours."""
        elif event is MarketEvent.POST_CLOSE or event is MarketEvent.REG_CLOSE:
            is_post = "including" if event is MarketEvent.POST_CLOSE else "excluding"
            prompt = f"""Provide the next weekday's close {is_post} post_market hours."""
        response = self.get_exchanges_csv(
            f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

            Weekdays are: Mon, Tue, Wed, Thu, Fri.
            On weekdays all exchanges open after pre-market and regular hours.
            On weekdays all exchanges close after regular and post-market hours.
            
            Weekends are: Sat, Sun.
            Always exclude weekends from exchange operating hours.
            Always exclude holidays from exchange operating hours.
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with a date that uses this format: '%a %b %d %X %Y'.""").text
        progress.update(1)
        return response

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            del self.events[exchange_code]
            return self.generated_events(exchange_code)
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE)) 
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_temp, 
                             contents=prompt)

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wikipedia Search Tool

In [15]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Gemini.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return StopGeneration().result

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Google Search Tool

In [16]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return StopGeneration().result # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest API Tool and Helpers

In [17]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",60
    POLY = "polygon.io",5 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [18]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(self.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def get(self, url: str) -> Optional[str]:
        try:
            request = requests.get(url)
            if request.status_code != requests.codes.ok:
                print(f"the endpoint returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = self.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.80
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return StopGeneration().result

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return StopGeneration().result

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return StopGeneration().result

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

# Instantiate the Tools

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [19]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [20]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays.
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends.
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours.
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [21]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 1000 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "published_utc.gte", "published_utc.lte", "order", "limit", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calling Expert

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

## Callable Functions and Handler

In [22]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != StopGeneration().result:
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return StopGeneration().result
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

## Define the System Prompt

In [23]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

## Import the Rest API Keys

In [24]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

## The Function Caller

In [25]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]

    # Gemini's innate notion of current date and time is unstable.
    contents += f"""
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them. Think step by step.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    response = api.retriable(api.client.models.generate_content,
                             model=api(Gemini.Model.GEN),
                             config=config_fncall,
                             contents=contents)
    while function_calling_in_process:
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
                # Send the updated prompt.
                response = api.retriable(api.client.models.generate_content,
                                         model=api(Gemini.Model.GEN),
                                         config=config_fncall,
                                         contents=contents)
            else:
                # Response may be a summary or reasoning step.
                if len(response.candidates[0].content.parts) == 1:
                    function_calling_in_process = False
                    break # No more parts in response.
                else:
                    #display(Markdown("#### Natural language reasoning step"))
                    #print(response)
                    continue # Next part contains a function call.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\$")))

api.refill_rpm  15


# Ask a question

<span style="font-size:18px;">
    If you're on free-tier of Gemini you probably want to Run-before here. Your usage tier can be configured in the api-helper at the top of the notebook.
</span>

In [26]:
send_message("What is the current session for US exchanges?")

Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

The current market session for US exchanges is closed.


In [27]:
send_message("What is the US market status?")

#### Natural language response

The U.S. market is currently closed. The timestamp for the market status is Fri Jun 20 00:36:05 2025 America/New_York time. There is no holiday today. The market session is closed.


In [29]:
send_message("When was the last US market close?")

#### Natural language response

The last market close in the United States was on Thursday, June 19, 2025, at 8:00 PM.


In [31]:
send_message("What is Apple's stock ticker?")

#### Natural language response

The stock ticker for Apple is AAPL.


In [35]:
send_message("What is the current price of Amazon stock? Use markdown formatting.")

#### Natural language response

The current price of Amazon (AMZN) stock is \$212.52. This information was last updated on Thu Jun 19 2025 at 16:00:00.

*   **Current Price:** \$212.52
*   **Change:** \-2.3
*   **Percent Change:** \-1.0707%
*   **High Price of the Day:** \$217.96
*   **Low Price of the Day:** \$212.34
*   **Open Price of the Day:** \$215.09
*   **Previous Close Price:** \$214.82


api.refill_rpm  15


In [36]:
send_message("Show me Apple's basic financials. How has the stock performed?")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Apple's (AAPL) basic financials and stock performance:

**Financial Highlights:**

*   **Profitability:** Apple has a trailing twelve month net profit margin of 24.3%.
*   **Gross Margin:** Apple's gross margin is 46.63% for the trailing twelve months.
*   **Revenue Growth:** Apple's revenue has grown 4.91% year-over-year for the trailing twelve months.
*   **Earnings per Share (EPS):** Apple's EPS is \$6.4078 for the trailing twelve months.
*   **Debt:** The Long Term Debt to Equity Annual is 1.5057.
*   **Dividends:** Apple's current dividend yield is 0.5215% (TTM), with a dividend per share of \$1.0115 (TTM).
*   **Valuation:**
    *   The price-to-earnings ratio is 30.1774 (TTM).
    *   The price-to-book ratio is 43.956.
    *   The price-to-sales ratio is 7.3335 (TTM).
*   **Return on Equity (ROE):** Apple's ROE is 151.31% for the trailing twelve months.
*   **52 Week Performance:**
    *   52-week high: \$260.1 on 2024-12-26
    *   52-week low: \$169.2101 on 2025-04-08
*   **Trading Volume:** The 10-day average trading volume is 51.26078, and the 3-month average is 57.68244.
*   **Stock Performance:**
    *   5-day price return: -1.3153%
    *   13-week price return: -7.5744%
    *   26-week price return: -20.6603%
    *   52-week price return: -9.2722%
    *   Year-to-date price return: -21.4999%
    *   Month-to-date price return: -2.126%



In [37]:
send_message("I need Apple's daily candlestick from 2025-05-05")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is a summary of Apple's daily candlestick data from May 5th, 2025:

*   **Symbol:** AAPL
*   **Open Price:** 203.1
*   **High Price:** 204.1
*   **Low Price:** 198.21
*   **Close Price:** 198.89
*   **Volume:** 69018452
*   **Pre-Market Price:** 205.0
*   **After-Hours Price:** 198.6
*   **Date:** 2025-05-05

In [39]:
send_message("Tell me who are Apple's peers?")

#### Natural language response

The peers of Apple, with ticker symbol AAPL, include Dell Technologies -C (DELL), Super Micro Computer Inc (SMCI), Hewlett Packard Enterprise (HPE), HP Inc (HPQ), Western Digital Corp (WDC), NetApp Inc (NTAP), Pure Storage Inc - Class A (PSTG), and IonQ Inc (IONQ). These companies operate in the same country and subIndustry as Apple.


In [41]:
send_message("Tell me the recommendation trends for all of Apple's peers")

api.on_error.next_model: model is now  gemini-2.0-flash-exp


#### Natural language response

Recommendation Trends for Apple's Peers:

Here's a summary of the latest analyst recommendation trends for Apple's peers, based on data up to June 2025:

*   **Dell Technologies (DELL):** The recommendation trend is consistent from March to June 2025. It shows a majority of "Buy" recommendations. For June 2025, there are 20 "Buy", 4 "Hold", 0 "Sell", 6 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **Super Micro Computer Inc (SMCI):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 10 "Buy", 10 "Hold", 2 "Sell", 2 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **Hewlett Packard Enterprise (HPE):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 7 "Buy", 8 "Hold", 0 "Sell", 4 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **HP Inc (HPQ):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 4 "Buy", 13 "Hold", 1 "Sell", 2 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **Western Digital Corp (WDC):** The recommendation trend shows an increase in "Buy" recommendations in June 2025. For June 2025, there are 19 "Buy", 5 "Hold", 0 "Sell", 6 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **NetApp Inc (NTAP):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 9 "Buy", 16 "Hold", 0 "Sell", 3 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **Pure Storage Inc - Class A (PSTG):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 13 "Buy", 6 "Hold", 1 "Sell", 8 "Strong Buy", and 0 "Strong Sell" recommendations.
*   **IONQ Inc (IONQ):** The recommendation trend is consistent from March to June 2025. For June 2025, there are 7 "Buy", 2 "Hold", 0 "Sell", 2 "Strong Buy", and 0 "Strong Sell" recommendations.


In [44]:
send_message("Tell me who are Amazon's peers?")

#### Natural language response

The peers of Amazon, based on the subIndustry grouping, are Coupang Incorporated (CPNG), Ebay Incorporated (EBAY), Ollie's Bargain Outlet Holdings (OLLI), Dillards Incorporated-Class A (DDS), Etsy Incorporated (ETSY), Macy's Incorporated (M), Savers Value Village Incorporated (SVV), and Groupon Incorporated (GRPN).


In [43]:
api.set_default_model(1) # generate with gemini-2.0-flash-exp
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.
    Discuss and provide details about any patterns you notice in the price data.""")
api.set_default_model(0)

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Amazon's (AMZN) stock information:

**Current Share Price:**

*   The current share price is \$212.52 as of June 19, 2025, at 04:00:00 PM Eastern Time.
*   This represents a decrease of \$2.30, or 1.07%, from the previous close.
*   The high of the day was \$217.96, and the low was \$212.34.
*   The opening price for the day was \$215.09.
*   The previous close price was \$214.82.

**Candlestick Data for the Past Month (May 19, 2025 - June 19, 2025):**

I am including a table of candlestick data for Amazon (AMZN) from May 19, 2025, to June 19, 2025. The data is sorted in descending order by date.

| Date       | Open    | High    | Low     | Close   | Volume      |
| :--------- | :------ | :------ | :------ | :------ | :---------- |
| 2025-06-19 | \$215.09 | \$217.96 | \$212.34 | \$212.52 | 44,360,509  |
| 2025-06-18 | \$215.20 | \$217.41 | \$214.56 | \$214.82 | 32,086,262  |
| 2025-06-17 | \$212.31 | \$217.06 | \$211.60 | \$216.10 | 33,284,158  |
| 2025-06-13 | \$209.96 | \$214.05 | \$209.62 | \$212.10 | 29,337,763  |
| 2025-06-12 | \$211.78 | \$213.58 | \$211.33 | \$213.24 | 27,639,991  |
| 2025-06-11 | \$217.41 | \$218.40 | \$212.89 | \$213.20 | 39,325,981  |
| 2025-06-10 | \$216.78 | \$217.69 | \$214.15 | \$217.61 | 31,303,317  |
| 2025-06-09 | \$214.75 | \$217.85 | \$212.88 | \$216.98 | 38,102,502  |
| 2025-06-04 | \$212.40 | \$213.87 | \$210.50 | \$213.57 | 39,832,500  |
| 2025-06-03 | \$209.55 | \$212.81 | \$207.56 | \$207.91 | 51,979,243  |
| 2025-06-02 | \$206.55 | \$208.18 | \$205.18 | \$207.23 | 29,915,592  |
| 2025-05-30 | \$207.11 | \$208.95 | \$205.03 | \$205.71 | 33,139,121  |
| 2025-05-29 | \$204.98 | \$207.00 | \$202.68 | \$206.65 | 29,113,319  |
| 2025-05-28 | \$204.84 | \$205.99 | \$201.70 | \$205.01 | 51,679,406  |
| 2025-05-27 | \$208.03 | \$208.81 | \$204.23 | \$205.70 | 34,700,005  |
| 2025-05-23 | \$205.92 | \$207.66 | \$204.41 | \$204.72 | 28,549,753  |
| 2025-05-22 | \$203.09 | \$206.69 | \$202.19 | \$206.02 | 34,892,044  |
| 2025-05-21 | \$198.90 | \$202.37 | \$197.85 | \$200.99 | 33,393,545  |
| 2025-05-20 | \$201.38 | \$205.76 | \$200.16 | \$203.10 | 38,938,882  |
| 2025-05-19 | \$201.61 | \$203.46 | \$200.06 | \$201.12 | 42,460,924  |
| 2025-05-16 | \$204.63 | \$205.59 | \$202.65 | \$204.07 | 29,470,373  |
| 2025-05-15 | \$201.65 | \$206.62 | \$201.26 | \$206.16 | 34,314,810  |

**Observations:**

*   **Volatility:** The stock price has experienced volatility over the past month, with noticeable fluctuations in the daily high and low prices.
*   **Downward Trend:** The closing price on June 19, 2025, is lower than the closing price a month prior on May 19, 2025, indicating a slight downward trend over the observed period.
*   **Volume Spikes:** There are a few days with significantly higher trading volumes, which could be associated with specific news events or market sentiment changes. For example, June 3, 2025, had a volume of 51,979,243.
*   **Recent Decline:** In the most recent days (June 17-19), the stock has shown a decline, closing at \$212.52 on June 19, 2025.



In [46]:
send_message("What is Apple's ticker overview")

#### Natural language response

Apple Incorporated (AAPL) is a major global company offering a wide range of hardware and software for consumers and businesses. The iPhone accounts for most of Apple's sales, while other products like Mac, iPad, and Watch are integrated into its software ecosystem. Apple is expanding into areas like streaming, subscriptions, and augmented reality. The company designs its own software and semiconductors, working with subcontractors such as Foxconn and TSMC for manufacturing. A little under half of Apple's sales are direct, through flagship stores, with the majority through partnerships and distribution.

Key details:
*   **Primary Exchange:** XNAS
*   **CIK:** 0000320193
*   **Composite FIGI:** BBG000B9XRY4
*   **Share Class FIGI:** BBG001S5N8V8
*   **Market Capitalization:** \$2,975,216,539,200.0
*   **Phone Number:** (408) 996-1010
*   **Address:** One Apple Park Way, Cupertino, CA 95014
*   **SIC Code:** 3571 (Electronic Computers)
*   **Homepage URL:** <https://www.apple.com>
*   **Total Employees:** 164,000
*   **List Date:** 1980-12-12
*   **Logo URL:** <https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_logo.svg>
*   **Icon URL:** <https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_icon.png>
*   **Weighted Shares Outstanding:** 14,935,826,000
*   **Round Lot:** 100

In [48]:
send_message("Tell me about Amazon's historical and current recommendation trends")

#### Natural language response

As of June 1st, 2025, Amazon has 24 strong buy, 51 buy, and 6 hold recommendations. There are no sell or strong sell recommendations.

Here's a summary of Amazon's recommendation trends:
*   June 1st, 2025: 24 strong buy, 51 buy, 6 hold
*   May 1st, 2025: 22 strong buy, 51 buy, 6 hold
*   April 1st, 2025: 23 strong buy, 50 buy, 4 hold
*   March 1st, 2025: 21 strong buy, 51 buy, 5 hold


In [49]:
send_message("What is Google's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Google's stock ticker symbols on the NASDAQ are GoogL and Goog. It is also listed on the Frankfurt Stock Exchange under the symbol GGQ1. These symbols now represent Alphabet Inc., Google's holding company.


api.zero_error: model is now  gemini-2.0-flash


In [57]:
send_message("What is MGM Studio's stock symbol?")

api.on_error.next_model: model is now  gemini-2.0-flash-001


#### Natural language response

I was unable to find the stock symbol for Mgm Studios.


api.zero_error: model is now  gemini-2.0-flash


In [55]:
send_message("What is Amazon MGM Studio's stock symbol?")

#### Natural language response

Amazon MGM Studios is a subsidiary of Amazon, which is a publicly traded company. The stock symbol for Amazon is AMZN.


In [58]:
send_message("What is Facebook's stock ticker symbol?")

Score similarity to stored grounding: 0it [00:00, ?it/s]


#### Natural language response

Facebook, now known as Meta Platforms, Inc., has the stock ticker symbol META. It was formerly FB.


In [59]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

As of June 20, 2025, here's a summary of Amazon's analyst recommendations and recent news sentiment:

**Analyst Recommendation Trends:**

*   The latest analyst recommendation trends show a strong bullish sentiment. In June 2025, out of the analysts providing recommendations, 24 rated the stock as a 'strong buy' and 51 rated it as a 'buy', with 6 analysts recommending to 'hold'. There were no 'sell' or 'strong sell' recommendations.

**Recent News and Sentiment Analysis:**

Recent news articles (May 20, 2025 - June 19, 2025) present a generally positive outlook for Amazon:

*   **AI and Cloud Computing:** Several articles highlight Amazon's strong position in the artificial intelligence (AI) and cloud computing sectors, particularly through Amazon Web Services (AWS). Analysts predict significant growth for AWS, driven by the increasing demand for AI infrastructure.
*   **Analyst Upgrades and Positive Ratings:** Multiple articles mention analysts reiterating bullish ratings on Amazon, with some suggesting the stock is poised for further gains.
*   **Robotics and Automation:** Amazon's increasing use of robotics and automation in its fulfillment centers is seen as a positive factor, leading to productivity gains and margin improvement.
*   **E-commerce Dominance:** Amazon continues to dominate the e-commerce market, with a significant market share in the U.S.
*   **Partnerships and Expansion:** Amazon's partnership with Roku for advertising and its potential tie-up with AST SpaceMobile are viewed favorably.
*   **Bill Ackman's Investment:** Billionaire investor Bill Ackman's Pershing Square has taken a new stake in Amazon, adding to the bullish sentiment around the stock.
*   **Potential Risks:** Some articles mention potential risks, such as tariffs and competition, but the overall sentiment remains positive due to Amazon's strong fundamentals and growth prospects.

**Overall Summary:**

The analyst recommendation trends and recent news sentiment suggest a predominantly bullish outlook for Amazon. Analysts are optimistic about the company's growth prospects, particularly in AI, cloud computing, and e-commerce. Recent news articles highlight positive developments, such as analyst upgrades, strategic partnerships, and investments in AI and automation. While potential risks exist, the overall sentiment indicates that Amazon is well-positioned for continued growth and success.


In [63]:
send_message(
    '''Tell me about Google's share price from May 01 2025 until today in a markdown table.
    How has the stock performed?
    Perform a sentiment analysis of news during the same dates. Include trends in your analysis.''')

Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's stock performance and news sentiment from May 1, 2025, to June 19, 2025.

**Stock Performance**

The following table summarizes Google's (GOOG) share price from May 1, 2025, to June 19, 2025.

| Date             | Open   | High   | Low    | Close  | Volume     |
| :--------------- | :----- | :----- | :----- | :----- | :--------- |
| Thu May 01 2025  | 162.52 | 163.94 | 160.93 | 162.79 | 21904291   |
| Fri May 02 2025  | 164.955| 166.7  | 163.66 | 165.81 | 16844937   |
| Mon May 05 2025  | 164.515| 167.1  | 164.47 | 166.05 | 15309343   |
| Tue May 06 2025  | 163.96 | 166.74 | 163.13 | 165.2  | 10691949   |
| Wed May 07 2025  | 166.07 | 166.99 | 149.49 | 152.8  | 78900429   |
| Thu May 08 2025  | 155.92 | 157.41 | 154.1  | 155.75 | 38387507   |
| Fri May 09 2025  | 155.55 | 156.43 | 153.83 | 154.38 | 22871035   |
| Mon May 12 2025  | 159.1  | 160.44 | 157.889| 159.58 | 31884901   |
| Tue May 13 2025  | 159.92 | 162.06 | 157.58 | 160.89 | 24944270   |
| Wed May 14 2025  | 161.31 | 168.34 | 160.93 | 166.81 | 31769209   |
| Thu May 15 2025  | 167.14 | 167.51 | 163.84 | 165.4  | 22717554   |
| Fri May 16 2025  | 168.93 | 170.65 | 166.95 | 167.43 | 36271378   |
| Mon May 19 2025  | 165.715| 167.95 | 165.415| 167.87 | 21374688   |
| Tue May 20 2025  | 167.76 | 169.68 | 164.26 | 165.32 | 33563274   |
| Wed May 21 2025  | 168.865| 169.8  | 166.68 | 167.71 | 25386713   |
| Thu May 22 2025  | 169.01 | 171.062| 168.65 | 170.37 | 24742877   |
| Fri May 23 2025  | 170.28 | 171.205| 169.26 | 169.59 | 24963648   |
| Tue May 27 2025  | 173.35 | 178.13 | 171.88 | 171.98 | 45024081   |
| Wed May 28 2025  | 173.98 | 176.48 | 173.014| 173.38 | 25999228   |
| Thu May 29 2025  | 175.0  | 175.4  | 171.78 | 172.96 | 21233590   |
| Fri May 30 2025  | 172.41 | 173.44 | 168.525| 172.85 | 36258254   |
| Mon Jun 02 2025  | 177.28 | 177.823| 172.84 | 173.98 | 32531762   |
| Mon Jun 02 2025  | 172.3  | 175.83 | 172.3  | 174.92 | 22258115   |
| Tue Jun 03 2025  | 173.58 | 178.343| 173.57 | 175.88 | 20873241   |
| Wed Jun 04 2025  | 175.87 | 177.915| 175.66 | 177.63 | 18817587   |
| Thu Jun 05 2025  | 177.48 | 178.13 | 176.11 | 176.97 | 17345924   |
| Fri Jun 06 2025  | 177.0  | 178.715| 175.94 | 177.23 | 17656119   |
| Mon Jun 09 2025  | 181.23 | 181.75 | 178.0  | 178.79 | 18994398   |
| Tue Jun 10 2025  | 177.76 | 182.445| 176.475| 180.01 | 32908000   |
| Wed Jun 11 2025  | 171.62 | 172.36 | 169.35 | 169.81 | 25422883   |
| Thu Jun 12 2025  | 168.28 | 169.58 | 167.795| 169.39 | 18508735   |
| Fri Jun 13 2025  | 168.93 | 170.65 | 166.95 | 167.43 | 36271378   |
| Mon Jun 16 2025  | 173.98 | 176.48 | 173.014| 173.38 | 25999228   |
| Tue Jun 17 2025  | 171.3  | 174.29 | 171.21 | 173.98 | 24341333   |

Overall, the stock experienced volatility during this period. It began at 162.79 on May 1, peaked at 180.01 on June 10, and closed at 173.98 on June 17.

**News Sentiment Analysis**

The news sentiment surrounding Google during this period was predominantly positive, with a focus on Google's advancements and investments in artificial intelligence (AI). Key themes include:

*   **AI Leadership:** Google is consistently recognized as a leader in AI, with its Gemini models and AI-powered search capabilities frequently mentioned.
*   **Cloud Growth:** Google Cloud Platform (GCP) is experiencing significant growth, driven by AI and cloud migration trends.
*   **Quantum Computing:** Google's Willow quantum computing chip is highlighted as a breakthrough, outperforming competitors.
*   **Partnerships:** Google is forming strategic partnerships, particularly in the autonomous vehicle sector with companies like Uber.
*   **Positive Analyst Ratings:** Many analysts recommend Google as a strong buy, citing its attractive valuation and growth potential.

However, some negative sentiments also emerged:

*   **Competition:** Google faces increasing competition from other tech giants in AI, cloud computing, and search.
*   **Regulatory Concerns:** Antitrust issues and potential regulations pose challenges to Google's business model.
*   **Tariff Impact:** Concerns about the impact of tariffs on Google's advertising revenue and supply chain are present.
*   **Water Usage:** Concerns about the water usage of data centers operated by Google.

**Concise Summary**

From May 1, 2025, to June 19, 2025, Google's stock price experienced volatility but generally trended upwards. News sentiment was largely positive, driven by Google's leadership in AI, cloud growth, and strategic partnerships. However, concerns about competition, regulatory issues, and the impact of tariffs also surfaced. Overall, the period reflects a company with strong fundamentals and growth potential, but also facing challenges in a rapidly evolving market.


In [65]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment from May 01 2025 until today?
    Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.''')

Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


api.on_error.next_model: model is now  gemini-2.0-flash-exp


Add chunks embedding: 0it [00:00, ?it/s]


limited 5/min, waiting 3.8723161220550537s


Add chunks embedding: 0it [00:00, ?it/s]


api.zero_error: model is now  gemini-2.0-flash


Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of the outlook for Apple and its peers based on recommendation trends and news sentiment from May 1, 2025, to June 20, 2025.

**Apple (AAPL):**

*   **Recommendation Trends:** Analyst recommendations for Apple have remained relatively stable, with a mix of "Buy" and "Hold" ratings. In June 2025, the recommendations were 14 strong buy, 25 buy, 14 hold, 3 sell, and 1 strong sell.
*   **News Sentiment:** The news sentiment surrounding Apple is mixed. Some articles highlight Apple's strong brand, customer loyalty, and potential in areas like AI and the smart home. However, there are also concerns about the company falling behind in the AI race, potential tariff impacts, and slowing iPhone sales growth. Several articles mention that Apple may need to acquire an AI platform to remain competitive.
*   **Peers:** Apple's peers by sub-industry include Dell Technologies, Super Micro Computer, Hewlett Packard Enterprise, HP Inc., Western Digital, NetApp, Pure Storage, and IonQ.

**Peer Analysis:**

Here's a summary of the recommendation trends and news sentiment for Apple's peers:

*   **Dell Technologies (DELL):** Recommendation trends are mostly positive. News sentiment is also positive, with analysts increasing price targets after recent earnings reports and strong demand for AI servers.
*   **Super Micro Computer (SMCI):** Recommendation trends are mixed, with a combination of "Buy" and "Hold" ratings. News sentiment is also mixed, with some articles highlighting the company's strong fundamentals and potential in the AI hardware market, while others express concerns about accounting issues and declining margins.
*   **Hewlett Packard Enterprise (HPE):** Recommendation trends are mostly "Hold." News sentiment is generally positive, with the company being recognized as a key player in the AI and data center equipment markets.
*   **HP Inc. (HPQ):** Recommendation trends are mostly "Hold." News sentiment is mixed, with some articles highlighting positive developments like dividend declarations and inclusion in retail events, while others point to mixed earnings and a cut in outlook.
*   **Western Digital (WDC):** Recommendation trends are mostly "Buy." News sentiment is neutral, with the company being mentioned as a key player in the storage systems market.
*   **NetApp (NTAP):** Recommendation trends are mostly "Hold." News sentiment is neutral, with the company being mentioned as a key player in the storage systems market.
*   **Pure Storage (PSTG):** Recommendation trends are mostly "Buy." News sentiment is neutral, with the company being mentioned as a key player in the storage systems market.
*   **IonQ (IONQ):** Recommendation trends are mostly "Buy." News sentiment is mixed, with some articles highlighting the company's potential in the quantum computing market, while others express concerns about its high valuation and uncertain timeline for practical applications.

**Concise Summary:**

The outlook for Apple is mixed. While the company has a strong brand and loyal customer base, there are concerns about its AI capabilities and potential tariff impacts. Apple's peers in the computer hardware sub-industry have varying outlooks, with some, like Dell, showing strong growth potential in AI, while others, like HP Inc., face challenges in certain segments. The quantum computing peer, IonQ, is considered high risk, high reward. Overall, the computer hardware sub-industry appears to be benefiting from the growth in AI and data center infrastructure, but individual company performance varies.


api.refill_rpm  15


In [66]:
api.set_default_model(1) # generate with gemini-2.0-flash-exp
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? From 2025-03-01 up to today.
    Also locate candlestick data for the same dates. 
    Discuss in detail any correlations in patterns between the candlestick and news data.
    Ignore duplicate news entry.''')
api.set_default_model(0)

Add chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of the news and candlestick data for Apple (AAPL) between March 1, 2025, and June 19, 2025, incorporating correlations between the two:

**News Summary:**

*   **Tariff Concerns:** Throughout the period, there were recurring concerns about potential tariffs on Apple products, particularly those manufactured in China and India. President Trump's threats to impose tariffs on iPhones manufactured outside the U.S. created uncertainty and negatively impacted the stock. Some analysts believed Apple could pass tariff costs to consumers, while others worried about increased costs and reduced competitiveness.
*   **AI Developments:** Apple's AI strategy was a significant theme. News articles highlighted concerns that Apple was falling behind in the AI race compared to competitors like Google and Microsoft. Some analysts suggested Apple might need to acquire an AI platform like Perplexity to remain competitive. There was also discussion about Apple's potential integration of AI into its Safari search capabilities, which could challenge Google's dominance.
*   **Market Performance & Analyst Ratings:** Apple's stock experienced volatility, with some analysts suggesting it was a buying opportunity during dips. However, other analysts expressed caution due to slowing growth, lack of innovation, and a high valuation. Several articles mentioned Apple as a top holding in various Vanguard ETFs, indicating its importance in the tech sector.
*   **Other Factors:** Other news included discussions about Apple's dividend yield, potential new product launches (like the HomePad and AI-powered robots), and its performance in specific markets like China and India.

**Candlestick Data Summary:**

*   The candlestick data shows price fluctuations throughout the period. There were periods of upward and downward trends, reflecting the uncertainty and volatility surrounding Apple's stock.
*   From March 1, 2025, to mid-March, the price generally declined.
*   In late March and early April, the price showed a recovery.
*   From mid-April to mid-May, the price experienced a significant decline.
*   In late May and June, the price showed a recovery.

**Correlations:**

*   **Tariff Announcements and Price Drops:** News of potential or actual tariff implementations often correlated with price drops in Apple's stock. For example, the announcement of a 25% tariff on iPhones manufactured in India on May 27, 2025, was followed by a price decrease.
*   **AI News and Market Sentiment:** Negative news regarding Apple's AI efforts or competitive positioning seemed to coincide with periods of market uncertainty or downward trends in the stock price. Conversely, positive news about potential AI partnerships or product innovations may have contributed to price stability or slight increases.
*   **Broader Market Trends:** Apple's stock performance was also influenced by broader market trends and investor sentiment. For example, the overall market sell-off in early May 2025, driven by concerns about rising interest rates and economic growth, impacted Apple's stock price along with other tech companies.
*   **Analyst Ratings:** Positive analyst ratings and price target increases may have provided some support to the stock price during periods of uncertainty.

**Concise Summary:**

Between March and June 2025, Apple's stock was significantly influenced by tariff-related news and its perceived position in the AI race. Negative tariff announcements and concerns about Apple's AI progress often correlated with price declines, while positive news or analyst ratings could provide some support. Broader market trends and investor sentiment also played a role in Apple's stock performance during this period.

**Disclaimer:** I am an AI chatbot and cannot provide financial advice. This information is for informational purposes only and should not be considered investment advice.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>

# Update June 7, 2025

<span style="font-size:18px;">
    Bugfix version 102 finally brings Essy to a stable milestone. A month and a half late :) There's still more to be built including adding reasoning, agents, and structured output. A few unimplemented rest endpoints remain that could make Essy more self-reliant. The vector store has gotten bigger but not smarter. Essy can tell us pre-scored news has some sentiment but cannot generate it due to limited summaries. Essy can detect interesting patterns in a dataset but not between adjacent datasets. There's so much data we'll need to recruit Essy some help.
</span>

# Advanced (localhost required)

<span style="font-size:18px;">
    The functions demonstrated here require a locally running notebook. A dedicated GPU with at least 8GB VRAM is recommended but not required. Output is generated with Gemma 3 12B QAT, Gemma.cpp, and (later) Gemma 3n. Output on Kaggle is based on cached data.
</span>

In [ ]:
# soon